In [1]:

import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
import awkward as ak

from sklearn.metrics import classification_report, roc_auc_score
from scipy import interpolate
from sklearn.metrics import roc_curve, auc
from cycler import cycler
# from awkward import ak

import uproot ## means uproot4
import sklearn.metrics as m

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)


CustNano = {
    "ttH(nonbb)"          : "/data/pubfs/zhaoyz/CustNano/2018/Signal_MERGED/ttHToNonbb_M125_TuneCP5_13TeV-powheg-pythia8_RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2/MERGED.root",
}
samples = {process : {} for process in CustNano}
for process in CustNano:
    samples[process] = uproot.lazy({CustNano[process]: "Events" }) 

/home/pku/zhaoyz/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
#Show the branching related to GEN:
for keys in samples["ttH(nonbb)"].fields:
    if "Gen" in keys: print(keys)

nGenJetAK15
GenJetAK15_eta
GenJetAK15_mass
GenJetAK15_phi
GenJetAK15_pt
nGenJetAK8
GenJetAK8_eta
GenJetAK8_mass
GenJetAK8_phi
GenJetAK8_pt
nGenJet
GenJet_eta
GenJet_mass
GenJet_phi
GenJet_pt
nGenPart
GenPart_eta
GenPart_mass
GenPart_phi
GenPart_pt
GenPart_genPartIdxMother
GenPart_pdgId
GenPart_status
GenPart_statusFlags
nGenSubJetAK15
GenSubJetAK15_eta
GenSubJetAK15_mass
GenSubJetAK15_phi
GenSubJetAK15_pt
nSubGenJetAK8
SubGenJetAK8_eta
SubGenJetAK8_mass
SubGenJetAK8_phi
SubGenJetAK8_pt
Generator_binvar
Generator_scalePDF
Generator_weight
Generator_x1
Generator_x2
Generator_xpdf1
Generator_xpdf2
Generator_id1
Generator_id2
GenVtx_x
GenVtx_y
GenVtx_z
nGenVisTau
GenVisTau_eta
GenVisTau_mass
GenVisTau_phi
GenVisTau_pt
GenVisTau_charge
GenVisTau_genPartIdxMother
GenVisTau_status
GenMET_phi
GenMET_pt
nGenDressedLepton
GenDressedLepton_eta
GenDressedLepton_mass
GenDressedLepton_phi
GenDressedLepton_pt
GenDressedLepton_pdgId
GenDressedLepton_hasTauAnc
nGenIsolatedPhoton
GenIsolatedPhoton_eta
G

### Here is an example for what I want to do:

In [3]:
#Show the lastcopy Higgs in each event.
islastcopyH = (np.bitwise_and(samples["ttH(nonbb)"]["GenPart_statusFlags"], (1<<13) ) > 0) & (samples["ttH(nonbb)"]["GenPart_pdgId"] == 25)
print("All events:",islastcopyH)
print("For example in 0th event:",islastcopyH[0])

All events: [[False, False, False, False, False, False, ... False, False, False, False, False]]
For example in 0th event: [False, False, False, False, False, False, ... False, False, False, False, False]


In [8]:
print("For example, the index(aka.PID) of last copy Higgs in 0th event is",np.where(islastcopyH[0] == True)[0][0])

For example, the index(aka.PID) of last copy Higgs in 0th event is 10


In [15]:
print("Then, the 2 daughter's pdgId of last copy Higgs in 0th event is",samples["ttH(nonbb)"]["GenPart_pdgId"][0][samples["ttH(nonbb)"]["GenPart_genPartIdxMother"][0] == np.where(islastcopyH[0] == True)[0][0]])
print("So the Higgs in 0th events decays into 2Ws.")

Then, the 2 daughter's pdgId of last copy Higgs in 0th event is [24, -24]
So the Higgs in 0th events decays into 2Ws.


### Bug occurs when we do this for all the events:

In [18]:
ak.where(islastcopyH == True ,highlevel = True)

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

### For the same reason we can't do:

In [19]:
daughterPDGid = samples["ttH(nonbb)"]["GenPart_pdgId"][samples["ttH(nonbb)"]["GenPart_genPartIdxMother"] == np.where(islastcopyH == True)[0][0]]

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)